In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
np.random.seed(1)

In [ ]:
def load_dataset():
    """
    @does: reads and returns the train and test set 
    """
    train_dataset = h5py.File('train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [ ]:
def convert_to_one_hot(Y, C):
    """
    @does: Convert the array to a one hot encoding
    """
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [ ]:
# Loading the dataset
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

OSError: ignored

In [ ]:
# As usual you flatten the image dataset, then normalize it by dividing by 255. On top of that, you will convert each label to a one-hot vector as shown in Figure 1. Run the cell below to do so.

# Flatten the training and test images
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T
# Normalize image vectors
X_train = X_train_flatten/255.
X_test = X_test_flatten/255.
# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6)
Y_test = convert_to_one_hot(Y_test_orig, 6)

print ("number of training examples = " + str(X_train.shape[1]))
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (12288, 1080)
Y_train shape: (6, 1080)
X_test shape: (12288, 120)
Y_test shape: (6, 120)


In [ ]:
def initializer():
    initializer = tf.initializers.GlorotNormal(seed=1)
    w1 = tf.Variable(initializer(shape = (25, 12288)), name = 'w1')
    w2 = tf.Variable(initializer(shape = (20, 25)), name = 'w2')
    w3 = tf.Variable(initializer(shape = (6, 20)), name = 'w3')
    
    b1 = tf.Variable(tf.zeros([25, 1]), name = 'b1')
    b2 = tf.Variable(tf.zeros([20, 1]), name = 'b2')
    b3 = tf.Variable(tf.zeros([6, 1]), name = 'b3')
    
    parameters = {
        'w1':w1,
        'w2':w2,
        'w3':w3,
        'b1':b1,
        'b2':b2,
        'b3':b3
    }
    return parameters
    

In [ ]:
parameters = initializer()

In [ ]:
parameters

{'w1': <tf.Variable 'w1:0' shape=(25, 12288) dtype=float32, numpy=
 array([[ 0.00183502, -0.00714326,  0.00914525, ..., -0.01221176,
          0.01656458,  0.00526992],
        [ 0.02264025,  0.0067227 ,  0.00795862, ...,  0.00284724,
          0.01910819,  0.00122853],
        [-0.00163333, -0.00861273, -0.01398893, ..., -0.00741132,
          0.02071251, -0.026417  ],
        ...,
        [-0.00126929,  0.01729332,  0.02082342, ...,  0.01709594,
          0.00429358, -0.00733263],
        [ 0.00275988,  0.00419503,  0.00949959, ...,  0.01241149,
         -0.0270652 ,  0.01501104],
        [-0.00145541,  0.02459595,  0.00339064, ..., -0.02478788,
          0.02716016, -0.00306428]], dtype=float32)>,
 'w2': <tf.Variable 'w2:0' shape=(20, 25) dtype=float32, numpy=
 array([[ 0.03035408, -0.11816052,  0.1512765 , -0.18906163, -0.21806385,
         -0.09566725, -0.00913596,  0.11216661, -0.00359232, -0.33624747,
          0.41090596, -0.33027324,  0.08969404,  0.16360672, -0.05161138,
    

In [ ]:
def forwardPass(X, y ,parameters):
    z1 = tf.add(tf.matmul(parameters['w1'], X), parameters['b1'])
    o1 = tf.nn.relu(z1)
    
    z2 = tf.add(tf.matmul(parameters['w2'], o1), parameters['b2'])
    o2 = tf.nn.relu(z2)
    
    z3 = tf.add(tf.matmul(parameters['w3'], o2), parameters['b3'])
    
    return z3

In [ ]:
def costFunction(z3, y):
    logits = tf.transpose(z3)## raw output of NNs
    labels = tf.transpose(y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    return cost

In [ ]:
def gradient(inputs, targets, parameters):
    with tf.GradientTape() as tape:
        z3 = forwardPass(inputs, targets, parameters)
        loss_value = costFunction(z3, targets)
        
    return [tape.gradient(loss_value, list(parameters.values())), loss_value]

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001)

for epoch in range(100):
    minibatches = random_mini_batches(X_train, Y_train, 32, seed = 1)
    
    for minibatch in minibatches:
        x = minibatch[0].astype(np.float32)
        y = minibatch[1].astype(np.float32)
        
        grads, loss_value = gradient(x, y, parameters)
        optimizer.apply_gradients(zip(grads, list(parameters.values())))

In [ ]:
optimizer.get_gradients

<bound method OptimizerV2.get_gradients of <keras.optimizer_v2.adam.Adam object at 0x28bcc1f40>>